# Assignment 9: GBDT

#### Response Coding: Example

<img src='http://i.imgur.com/TufZptV.jpg' width=700px>

> The response tabel is built only on train dataset.
> For a category which is not there in train data and present in test data, we will encode them with default values
Ex: in our test data if have State: D then we encode it as [0.5, 0.05]

<ol>
    <li><strong>Apply GBDT on these feature sets</strong>
        <ul>
            <li><font color='red'>Set 1</font>: categorical(instead of one hot encoding, try <a href='https://www.appliedaicourse.com/course/applied-ai-course-online/lessons/handling-categorical-and-numerical-features/'>response coding</a>: use probability values), numerical features + project_title(TFIDF)+  preprocessed_eassay (TFIDF)+sentiment Score of eassay(check the bellow example, include all 4 values as 4 features)</li>
            <li><font color='red'>Set 2</font>: categorical(instead of one hot encoding, try <a href='https://www.appliedaicourse.com/course/applied-ai-course-online/lessons/handling-categorical-and-numerical-features/'>response coding</a>: use probability values), numerical features + project_title(TFIDF W2V)+  preprocessed_eassay (TFIDF W2V)</li>        </ul>
    </li>
    <li><strong>The hyper paramter tuning (Consider any two hyper parameters)</strong>
        <ul>
    <li>Find the best hyper parameter which will give the maximum <a href='https://www.appliedaicourse.com/course/applied-ai-course-online/lessons/receiver-operating-characteristic-curve-roc-curve-and-auc-1/'>AUC</a> value</li>
    <li>find the best hyper paramter using k-fold cross validation/simple cross validation data</li>
    <li>use gridsearch cv or randomsearch cv or you can write your own for loops to do this task</li>
        </ul>
    </li>
    <li>
    <strong>Representation of results</strong>
        <ul>
    <li>You need to plot the performance of model both on train data and cross validation data for each hyper parameter, like shown in the figure
    <img src='https://i.imgur.com/Gp2DQmh.jpg' width=500px> with X-axis as <strong>n_estimators</strong>, Y-axis as <strong>max_depth</strong>, and Z-axis as <strong>AUC Score</strong> , we have given the notebook which explains how to plot this 3d plot, you can find it in the same drive <i>3d_scatter_plot.ipynb</i></li>
            <p style="text-align:center;font-size:30px;color:red;"><strong>or</strong></p> <br>
    <li>You need to plot the performance of model both on train data and cross validation data for each hyper parameter, like shown in the figure
    <img src='https://i.imgur.com/fgN9aUP.jpg' width=300px> <a href='https://seaborn.pydata.org/generated/seaborn.heatmap.html'>seaborn heat maps</a> with rows as <strong>n_estimators</strong>, columns as <strong>max_depth</strong>, and values inside the cell representing <strong>AUC Score</strong> </li>
    <li>You choose either of the plotting techniques out of 3d plot or heat map</li>
    <li>Once after you found the best hyper parameter, you need to train your model with it, and find the AUC on test data and plot the ROC curve on both train and test.
    <img src='https://i.imgur.com/wMQDTFe.jpg' width=300px></li>
    <li>Along with plotting ROC curve, you need to print the <a href='https://www.appliedaicourse.com/course/applied-ai-course-online/lessons/confusion-matrix-tpr-fpr-fnr-tnr-1/'>confusion matrix</a> with predicted and original labels of test data points
    <img src='https://i.imgur.com/IdN5Ctv.png' width=300px></li>
            </ul>
    <br>
    <li>You need to summarize the results at the end of the notebook, summarize it in the table format
        <img src='http://i.imgur.com/YVpIGGE.jpg' width=400px>
    </li>
</ol>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# import nltk
# nltk.download('vader_lexicon')

sid = SentimentIntensityAnalyzer()

for_sentiment = 'a person is a person no matter how small dr seuss i teach the smallest students with the biggest enthusiasm \
for learning my students learn in many different ways using all of our senses and multiple intelligences i use a wide range\
of techniques to help all my students succeed students in my class come from a variety of different backgrounds which makes\
for wonderful sharing of experiences and cultures including native americans our school is a caring community of successful \
learners which can be seen through collaborative student project based learning in and out of the classroom kindergarteners \
in my class love to work with hands on materials and have many different opportunities to practice a skill before it is\
mastered having the social skills to work cooperatively with friends is a crucial aspect of the kindergarten curriculum\
montana is the perfect place to learn about agriculture and nutrition my students love to role play in our pretend kitchen\
in the early childhood classroom i have had several kids ask me can we try cooking with real food i will take their idea \
and create common core cooking lessons where we learn important math and writing concepts while cooking delicious healthy \
food for snack time my students will have a grounded appreciation for the work that went into making the food and knowledge \
of where the ingredients came from as well as how it is healthy for their bodies this project would expand our learning of \
nutrition and agricultural cooking recipes by having us peel our own apples to make homemade applesauce make our own bread \
and mix up healthy plants from our classroom garden in the spring we will also create our own cookbooks to be printed and \
shared with families students will gain math and literature skills as well as a life long enjoyment for healthy cooking \
nannan'
ss = sid.polarity_scores(for_sentiment)

for k in ss:
    print('{0}: {1}, '.format(k, ss[k]), end='')

# we can use these 4 things as features/attributes (neg, neu, pos, compound)
# neg: 0.0, neu: 0.753, pos: 0.247, compound: 0.93

/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


LookupError: ignored

<h1>1. GBDT (xgboost/lightgbm) </h1>

## 1.1 Loading Data

In [132]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc

import re
# Tutorial about Python regular expressions: https://pymotw.com/2/re/

import pickle
from tqdm import tqdm
import os

#from chart_studio.plotly import plotly
#import chart_studio.plotly.offline as offline
#import chart_studio.plotlyplotly.graph_objs as go
#offline.init_notebook_mode()
from collections import Counter

In [133]:
import pandas
data = pandas.read_csv('preprocessed_data.csv')

In [134]:
data.head()

,school_state,teacher_prefix,project_grade_category,teacher_number_of_previously_posted_projects,project_is_approved,clean_categories,clean_subcategories,essay,price
0,ca,mrs,grades_prek_2,53,1,math_science,appliedsciences health_lifescience,i fortunate enough use fairy tale stem kits cl...,725.05
1,ut,ms,grades_3_5,4,1,specialneeds,specialneeds,imagine 8 9 years old you third grade classroo...,213.03
2,ca,mrs,grades_prek_2,10,1,literacy_language,literacy,having class 24 students comes diverse learner...,329.00
3,ga,mrs,grades_prek_2,2,1,appliedlearning,earlydevelopment,i recently read article giving students choice...,481.04
4,wa,mrs,grades_3_5,2,1,literacy_language,literacy,my students crave challenge eat obstacles brea...,17.74


<h2>1.2 Splitting data into Train and cross validation(or test): Stratified Sampling</h2>

In [135]:
# please write all the code with proper documentation, and proper titles for each subsection
# go through documentations and blogs before you start coding
# first figure out what to do, and then think about how to do.
# reading and understanding error messages will be very much helpfull in debugging your code
# when you plot any graph make sure you use 
    # a. Title, that describes your plot, this will be very helpful to the reader
    # b. Legends if needed
    # c. X-axis label
    # d. Y-axis label
Y=data["project_is_approved"]
X=data.drop(columns=["project_is_approved"])
print("Number of datapoints of each class label")
data['project_is_approved'].value_counts()

Number of datapoints of each class label


1    92706
0    16542
Name: project_is_approved, dtype: int64

In [136]:
from imblearn.over_sampling import RandomOverSampler
ros=RandomOverSampler(random_state=0)
X_resampled, y_resampled=ros.fit_resample(X,Y)
X=pd.DataFrame(X_resampled,columns=X.columns)
Y=y_resampled


In [137]:
from sklearn.model_selection import train_test_split
import math
xtrain,xtest,ytrain,ytest=train_test_split(X,Y,random_state=52,stratify=Y)

<h2>1.3 Make Data Model Ready: encoding eassay, and project_title</h2>

In [12]:
# please write all the code with proper documentation, and proper titles for each subsection
# go through documentations and blogs before you start coding
# first figure out what to do, and then think about how to do.
# reading and understanding error messages will be very much helpfull in debugging your code
# make sure you featurize train and test data separatly

# when you plot any graph make sure you use 
    # a. Title, that describes your plot, this will be very helpful to the reader
    # b. Legends if needed
    # c. X-axis label
    # d. Y-axis label
from sklearn.feature_extraction.text import TfidfVectorizer
def tfidf(train,test):
    tfidfvec=TfidfVectorizer(min_df=10,ngram_range=(1,4),max_features=5000)
    tfidfvec.fit(train['essay'])
    train_tfidf=tfidfvec.transform(train['essay'])
    test_tfidf=tfidfvec.transform(test['essay'])
    return(tfidfvec,train_tfidf,test_tfidf)
def tfidfw2v(train,test):
    with open('glove_vectors', 'rb') as f:
        model = pickle.load(f)
        glove_words =  set(model.keys())
    tfidfvec,train_tfidf,test_tfidf=tfidf(train,test)
    dictionary=dict(zip(tfidfvec.get_feature_names(),list(tfidfvec.idf_)))
    tfidf_words=set(tfidfvec.get_feature_names())
    tfidf_w2v_train=[]
    tfidf_w2v_test=[]
    for sentence in train['essay']:
        vector=np.zeros(300)
        tf_idf_weight=0
        for word in sentence.split():
            if (word in glove_words) and (word in tfidf_words):
                vec=model[word]
                tf_idf=dictionary[word]*(sentence.count(word)/len(sentence.split()))
                vector+=(vec*tf_idf)
                tf_idf_weight+=tf_idf
        if tf_idf_weight!=0:
            vector /= tf_idf_weight
        tfidf_w2v_train.append(vector)
    for sentence in test['essay']:
        vector=np.zeros(300)
        tf_idf_weight=0
        for word in sentence.split():
            if (word in glove_words) and (word in tfidf_words):
                vec=model[word]
                tf_idf=dictionary[word]*(sentence.count(word)/len(sentence.split()))
                vector+=(vec*tf_idf)
                tf_idf_weight+=tf_idf
        if tf_idf_weight!=0:
            vector /= tf_idf_weight
        tfidf_w2v_test.append(vector)
    return(tfidf_w2v_train,tfidf_w2v_test)
    




In [9]:
tfidf,train_tfidf,test_tfidf=tfidf(xtrain,xtest)

In [10]:
print(train_tfidf.shape)
print(test_tfidf.shape)

(139059, 5000)
(46353, 5000)


In [13]:
tfidf_w2v_train,tfidf_w2v_test=tfidfw2v(xtrain,xtest)

In [14]:
print(len(tfidf_w2v_train))
print(len(tfidf_w2v_train[0]))


139059
300


<h2>1.4 Make Data Model Ready: encoding numerical, categorical features</h2>

In [144]:
# please write all the code with proper documentation, and proper titles for each subsection
# go through documentations and blogs before you start coding 
# first figure out what to do, and then think about how to do.
# reading and understanding error messages will be very much helpfull in debugging your code
# make sure you featurize train and test data separatly

# when you plot any graph make sure you use 
    # a. Title, that describes your plot, this will be very helpful to the reader
    # b. Legends if needed
    # c. X-axis label
    # d. Y-axis label
from sklearn.feature_extraction.text import CountVectorizer
def responsecoding(col,data,ytrain):
    ztrain=ytrain.tolist()
    data_col=data[col].values
    vectorizer= CountVectorizer()
    text_bow = vectorizer.fit_transform(data_col)
    features_name=vectorizer.get_feature_names()
   
    
    dict={}
    for name in features_name:
        dict[name]={'1':0, '0':0}
    for i in range(len(data)):
        if ztrain[i]==1:
            dict[data.iloc[i][col]]['1']+=1
        else:
            dict[data.iloc[i][col]]['0']+=1
    for name in features_name:
        dict[name]['1']= dict[name]['1']/(dict[name]['1']+dict[name]['0'])
        dict[name]['0']=1-dict[name]['1']
    
    return(dict)

In [148]:
school_state_response_coding=responsecoding("school_state",xtrain,ytrain)
teacher_prefix_response_coding=responsecoding("teacher_prefix",xtrain,ytrain)
project_grade_category_response_coding=responsecoding("project_grade_category",xtrain,ytrain)
clean_categories_response_coding=responsecoding("clean_categories",xtrain,ytrain)
clean_subcategories_response_coding=responsecoding("clean_subcategories",xtrain,ytrain)

KeyError: 'literacy_language math_science'

In [151]:
xtrain['clean_categories'].value_counts()

literacy_language                       28531
math_science                            23477
literacy_language math_science          17650
health_sports                           12964
music_arts                               6527
specialneeds                             5904
appliedlearning                          5285
literacy_language specialneeds           5038
math_science literacy_language           2835
appliedlearning literacy_language        2690
history_civics                           2461
math_science specialneeds                2421
literacy_language music_arts             2236
math_science music_arts                  2135
appliedlearning specialneeds             2067
math_science appliedlearning             1605
health_sports specialneeds               1599
history_civics literacy_language         1581
appliedlearning math_science             1476
warmth care_hunger                       1323
appliedlearning music_arts               1043
health_sports literacy_language   

len(ytrain)

<br>Apply GBDT on different kind of featurization as mentioned in the instructions
<br> For Every model that you work on make sure you do the step 2 and step 3 of instrucations

In [ ]:
# please write all the code with proper documentation, and proper titles for each subsection
# go through documentations and blogs before you start coding
# first figure out what to do, and then think about how to do.
# reading and understanding error messages will be very much helpfull in debugging your code
# when you plot any graph make sure you use 
    # a. Title, that describes your plot, this will be very helpful to the reader
    # b. Legends if needed
    # c. X-axis label
    # d. Y-axis label

<h1>3. Summary</h1>

<br> as mentioned in the step 4 of instructions